In [ ]:
import os 
os.chdir('drive/MyDrive/entrepreneur-helper')

In [ ]:
ls

config.py         main.py     __pycache__/      text_handling.py
data_1.json       nn.py       requests.db       users.db
db_management.py  parsers.py  requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install wikipedia

In [ ]:
!python main.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Bot started!
Added thread: search_for_relevant_part_in_json
Added thread: parse_google
Added thread: wiki_parser
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:708: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:301: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)
Added thr

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


# Building NN for text relevancy estimation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import numpy as np
import warnings
warnings.simplefilter("ignore")
import torch.nn as nn
from scipy.special import softmax

In [ ]:
class RelevanceEstimator(nn.Module):
    '''
    NN module used for prediction of relevancy of corpus and given texts

    model_type: gradient boosting classifier / logistic reqression etc.
    X: train data
    y: train data labels
    batch_size: batch_size for train data

    '''
    def __init__(self,device:str, X:np.ndarray, y:np.ndarray, batch_size:int, 
                 model_type:str, model_kwargs:dict={}) -> None:
        super(RelevanceEstimator, self).__init__()

        if model_type == 'catboost':
            self.model = CatBoostClassifier(**model_kwargs)
        elif model_type == 'logreg':
            self.model = LogisticRegression(**model_kwargs)
        else:
            raise ValueError("Invalid model type")

        self.X = np.array(X)
        self.y = np.array(y)
        self.batch_size = batch_size
        self.reset_data = True

        self.pipeline = nn.Sequential(
            nn.BatchNorm1d(num_features=len(self.X[0])),
            nn.Softmax(),
        )

    @staticmethod
    def batch_normalization(vals:np.ndarray, eps=1e-5) -> np.ndarray:
        '''
        A function normalizing batch with respect to mean and varience

        vals: list of floats/ints meant to be normalized
        eps: constant meant to prevent division by zero
        '''
        mean_val = np.mean(vals)
        varience = np.var(vals)
        vals = (vals - mean_val) / np.sqrt(varience + eps)

        return vals  

    def normalize_and_softmax(self, vals) -> None:
        '''
        Batch preparation method
        '''
        for i in range(len(vals)):
            vals[i] = self.batch_normalization(vals[i])
            vals[i] = softmax(vals[i])


    def split_and_preprocess_data(self, random_state=42, test_size=0.3, shuffle=True) -> None:
        '''
        Method used for splitting X into train and test datasets
        '''
        self.train_data, self.test_data, self.train_labels, self.test_labels = \
        train_test_split(test_size=test_size, 
                         random_state=random_state,
                         shuffle=shuffle)
        
        print('Data successfully splitted!')


    def __batch_generator(self, dataset_type) -> tuple:
        '''
        Method used for data generation
        dataset_type: train or test
        '''
        if self.reset_data:
            try:
                self.train_generator = (self.train[i * self.batch_size : (i+1) * self.batch_size] for i in range(len(self.train) // self.batch_size - 1))
                self.test_generator = (self.test[i * self.batch_size : (i+1) * self.batch_size] for i in range(len(self.test) // self.batch_size - 1))
                self.train_labels_generator = (self.train_labels[i * self.batch_size : (i+1) * self.batch_size] for i in range(len(self.train_labels) // self.batch_size - 1))
                self.test_labels_generator = (self.test_labels[i * self.batch_size : (i+1) * self.batch_size] for i in range(len(self.test_labels) // self.batch_size - 1))

                self.reset_data = False

            except AttributeError:
                print("Seems like you haven't ran split_data method just yet")
                print("Aborting...")

        
        if dataset_type == 'train':
            return next(self.train_generator), next(self.train_labels_generator)
        elif dataset_type == 'test':
            return next(self.test_generator), next(self.test_labels_generator)
        else:
            raise ValueError("Invalid dataset type") 

    def fit(self, num_iters=1000):
        for _ in range(num_iters):
            for i in range(len(self.train) // self.batch_size - 1):
                X_batch, y_batch = self.__batch_generator("train")
                self.model.fit(X_batch, y_batch)

            self.reset_data = True

    def predict(self, X):
        apply_pipeline = self.pipeline(X)
        return self.model.predict(apply_pipeline.detach().numpy())


In [20]:
!git status
!git add entrepreneur-helper.ipynb

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	config.py
	data_1.json
	db_management.py
	entrepreneur-helper.ipynb
	main.py
	nn.py
	parsers.py
	requests.db
	requirements.txt
	text_handling.py
	users.db

nothing added to commit but untracked files present (use "git add" to track)


In [21]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   entrepreneur-helper.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	config.py
	data_1.json
	db_management.py
	main.py
	nn.py
	parsers.py
	requests.db
	requirements.txt
	text_handling.py
	users.db



In [25]:
!git config --global user.email "daniildiveev@mail.ru"
!git config --global user.name "sexozavr"

In [23]:
!git remote add origin https://github.com/sexozavr/entrepreneur-helper.git

In [26]:
!git commit -m '[ADD] added entrepreneur-helper.ipynb'

[master (root-commit) 7819c42] [ADD] added entrepreneur-helper.ipynb
 1 file changed, 1 insertion(+)
 create mode 100644 entrepreneur-helper.ipynb


In [31]:
!git log

commit 7819c426fabb0f4d4f79833efa3cbc9fb4825325 (HEAD -> master)
Author: sexozavr <daniildiveev@mail.ru>
Date:   Thu Apr 14 08:08:00 2022 +0000

    [ADD] added entrepreneur-helper.ipynb


In [29]:
!git push origin main

error: src refspec main does not match any.
error: failed to push some refs to 'https://github.com/sexozavr/entrepreneur-helper.git'


In [18]:
os.chdir('entrepreneur-helper')

In [30]:
!ls

config.py	  entrepreneur-helper.ipynb  parsers.py   requirements.txt
data_1.json	  main.py		     __pycache__  text_handling.py
db_management.py  nn.py			     requests.db  users.db
